In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
cust_prod = pd.read_csv('data/customer_product.csv')
cust_prod = cust_prod.drop(['Unnamed: 0'], axis=1)
cust_prod.head()

,customer_id,product,signup_date_time,cancel_date_time
0,C2448,prd_1,2017-01-01 10:35:09,NaN
1,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02
2,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55
3,C2451,prd_2,2017-01-01 13:32:08,NaN
4,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01


In [3]:
prod = pd.read_csv('data/product_info.csv')
prod

,product_id,name,price,billing_cycle
0,prd_1,annual_subscription,1200,12
1,prd_2,monthly_subscription,125,1


## Merge and Preprocess Subscription Data

- Extract Subscription Times
- Add Revenue Amounts
- Calculate Totals per year

In [6]:
cust_prod['signup_date_time'].max()

'2021-12-31 19:38:21'

In [11]:
subs = cust_prod.copy()
dummy_date = pd.Timestamp('2022-01-01 00:00:00')

# Convert sign-up and cancel dates to datetime format
subs['signup_date_time'] = pd.to_datetime(subs['signup_date_time'])
subs['cancel_date_time'] = pd.to_datetime(subs['cancel_date_time']).fillna(dummy_date)

# Create a new column to calculate subscription length in months
subs['sub_len_months'] = ((subs['cancel_date_time'] - subs['signup_date_time']) / pd.Timedelta(days=30))
subs['sub_len_years'] = subs['sub_len_months'] / 12

# TODO: update below additions to use numpy for speed
subs['active_flag'] = [1 if x==dummy_date else 0 for x in subs['cancel_date_time']]

subs['total_rev'] = [subs.iloc[x]['sub_len_months']*125 if subs.iloc[x]['product']=='prd_2' else subs.iloc[x]['sub_len_years']*1200 for x in range(len(subs))]

subs.head()

,customer_id,product,signup_date_time,cancel_date_time,sub_len_months,sub_len_years,active_flag,total_rev
0,C2448,prd_1,2017-01-01 10:35:09,2022-01-01 00:00:00,60.851964,5.070997,1,6085.196412
1,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02,56.931031,4.744253,0,5693.103125
2,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55,24.739882,2.061657,0,2473.988233
3,C2451,prd_2,2017-01-01 13:32:08,2022-01-01 00:00:00,60.847867,5.070656,1,7605.983410
4,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01,54.639086,4.553257,0,5463.908603


In [10]:
subs.active_flag.unique()

array([0], dtype=int64)